In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import tensorflow as tf
import datetime
from tensorflow.keras import utils
from keras.layers import Dense, Activation, Flatten, LSTM
from keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [2]:
data_2021 = pd.read_csv("données/valeursfoncieres-2022-s1.csv", sep="|")

C:\Users\xavid\AppData\Local\Temp\ipykernel_22120\3120857919.py:1: DtypeWarning: Columns (18,23,24,26,28,31,33,41) have mixed types. Specify dtype option on import or set low_memory=False.
  data_2021 = pd.read_csv("données/valeursfoncieres-2022-s1.csv", sep="|")


In [3]:
data_2021

,Code service sages,Reference document,1 Articles CGI,2 Articles CGI,3 Articles CGI,4 Articles CGI,5 Articles CGI,No disposition,Date mutation,Nature mutation,...,Surface Carrez du 5eme lot,Nombre de lots,Code type local,Type local,Identifiant local,Surface reelle bati,Nombre pieces principales,Nature culture,Nature culture speciale,Surface terrain
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,03/01/2022,Vente,...,NaN,1,2.0,Appartement,NaN,24.0,1.0,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,03/01/2022,Vente,...,NaN,0,NaN,NaN,NaN,NaN,NaN,S,NaN,84.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,03/01/2022,Vente,...,NaN,0,NaN,NaN,NaN,NaN,NaN,S,NaN,88.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,03/01/2022,Vente,...,NaN,1,2.0,Appartement,NaN,140.0,3.0,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,04/01/2022,Vente,...,NaN,0,NaN,NaN,NaN,NaN,NaN,T,NaN,510.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1429088,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,21/06/2022,Vente,...,NaN,2,3.0,Dépendance,NaN,0.0,0.0,NaN,NaN,NaN
1429089,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,21/06/2022,Vente,...,NaN,1,3.0,Dépendance,NaN,0.0,0.0,NaN,NaN,NaN
1429090,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,30/06/2022,Vente,...,NaN,1,2.0,Appartement,NaN,15.0,1.0,NaN,NaN,NaN
1429091,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,10/02/2022,Adjudication,...,NaN,1,3.0,Dépendance,NaN,0.0,0.0,NaN,NaN,NaN


In [4]:
data_2021_c = data_2021[['Nature mutation', 'Valeur fonciere', 'No voie', 'Code postal',
                         'Code departement', 'Code type local', 'Code voie', 'Code type local', 'Surface reelle bati', 'Surface terrain']]

In [5]:
data_2021_c.head()

,Nature mutation,Valeur fonciere,No voie,Code postal,Code departement,Code type local,Code voie,Code type local,Surface reelle bati,Surface terrain
0,Vente,"55000,00",13.0,1000.0,1,2.0,2280,2.0,24.0,NaN
1,Vente,"143000,00",NaN,1480.0,1,NaN,B010,NaN,NaN,84.0
2,Vente,"143000,00",NaN,1480.0,1,NaN,B010,NaN,NaN,88.0
3,Vente,"143000,00",98.0,1480.0,1,2.0,0055,2.0,140.0,NaN
4,Vente,"300,00",NaN,1480.0,1,NaN,B031,NaN,NaN,510.0


In [6]:
data_2021_c["Code departement"].unique()

array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
       21, 22, 23, 24, 25, 26, 27, 28, 29, '29', '2A', '2B', '30', 30, 31,
       32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48,
       49, 50, 51, 52, 53, 54, 55, 56, 58, 59, 60, 61, 62, 63, 64, 65, 66,
       69, 70, 71, 72, 73, 74, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86,
       87, 88, 89, 90, 91, 92, 93, 94, 95, 971, 972, 973, 974, 75],
      dtype=object)

In [7]:
data_2021_c.dropna(axis=0, inplace=True)
data_2021_c= data_2021_c.fillna(0)
data_2021_c = data_2021_c[data_2021_c['Nature mutation'] == 'Vente']
data_2021_c = data_2021_c.drop(columns='Nature mutation')

C:\Users\xavid\AppData\Local\Temp\ipykernel_22120\1905939202.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_2021_c.dropna(axis=0, inplace=True)


In [8]:
data_2021_c['Code postal'] = data_2021_c['Code postal'].astype(int)
data_2021_c['No voie'] = data_2021_c['No voie'].astype(int)
data_2021_c['Code postal'] = data_2021_c['Code postal'].astype(str)
data_2021_c['No voie'] = data_2021_c['No voie'].astype(str)
data_2021_c['Surface reelle bati'] = data_2021_c['Surface reelle bati'].astype(str)
data_2021_c["ID_Location"] = data_2021_c['Code voie'].astype(str) +"-"+ data_2021_c["Code postal"]+"-"+ data_2021_c["No voie"]+"-"+ data_2021_c["Surface reelle bati"]
data_2021_c['Code postal'] = data_2021_c['Code postal'].astype(float)
data_2021_c['No voie'] = data_2021_c['No voie'].astype(float)
data_2021_c['Code departement'] = data_2021_c['Code departement'].astype(str)
data_2021_c['Code departement'] = data_2021_c['Code departement'].str.replace('2A', '222')
data_2021_c['Code departement'] = data_2021_c['Code departement'].str.replace('2B', '223')
data_2021_c = data_2021_c.drop(columns='Code voie')

In [9]:
data_2021_c["Code departement"].unique()

array(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '21', '22', '23', '24',
       '25', '26', '27', '28', '29', '222', '223', '30', '31', '32', '33',
       '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44',
       '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55',
       '56', '58', '59', '60', '61', '62', '63', '64', '65', '66', '69',
       '70', '71', '72', '73', '74', '76', '77', '78', '79', '80', '81',
       '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92',
       '93', '94', '95', '971', '972', '973', '974', '75'], dtype=object)

In [10]:
data_2021_c['Valeur fonciere'] = data_2021_c['Valeur fonciere'].str.replace(',', '.')
data_2021_c['Valeur fonciere'] = data_2021_c['Valeur fonciere'].astype(float)
data_2021_c['Valeur fonciere'] = data_2021_c['Valeur fonciere'].astype(int)

data_2021_c['No voie'] = data_2021_c['No voie'].astype(int)
data_2021_c['Code postal'] = data_2021_c['Code postal'].astype(int)
data_2021_c['Code type local'] = data_2021_c['Code type local'].astype(int)
#data_2021_c['Surface terrain'] = data_2021_c['Surface terrain'].astype(int)
data_2021_c['Surface reelle bati'] = data_2021_c['Surface reelle bati'].str.replace(',', '.')
data_2021_c['Surface reelle bati'] = data_2021_c['Surface reelle bati'].astype(float)
data_2021_c['Surface reelle bati'] = data_2021_c['Surface reelle bati'].astype(int)

In [11]:
data_2021_c.set_index('ID_Location', inplace=True)

In [12]:
data_2021_c

,Valeur fonciere,No voie,Code postal,Code departement,Code type local,Code type local,Surface reelle bati,Surface terrain
ID_Location,,,,,,,,
0130-1560-282-108.0,255000,282,1560,1,1,1,108,649.0
0300-1390-217-126.0,525000,217,1390,1,2,2,126,628.0
0300-1390-217-424.0,525000,217,1390,1,4,4,424,628.0
0397-1000-6-100.0,140000,6,1000,1,1,1,100,796.0
0119-1500-45-204.0,580000,45,1500,1,1,1,204,496.0
...,...,...,...,...,...,...,...,...
3419-75016-10-0.0,52000000,10,75016,75,3,3,0,1005.0
3419-75016-10-0.0,52000000,10,75016,75,3,3,0,1005.0
3419-75016-10-0.0,52000000,10,75016,75,3,3,0,1005.0


In [13]:
data_2021_c.sort_values(by=['ID_Location'], ascending=True, inplace=True)

In [14]:
data_2021_c.dropna(axis=0, inplace=True)
data_2021_c= data_2021_c.fillna(0)
data_2021_c = data_2021_c.drop_duplicates()
data_2021_c

,Valeur fonciere,No voie,Code postal,Code departement,Code type local,Code type local,Surface reelle bati,Surface terrain
ID_Location,,,,,,,,
0001-10190-46-106.0,235000,46,10190,10,1,1,106,500.0
0001-10190-46-106.0,235000,46,10190,10,1,1,106,529.0
0001-10200-21-0.0,90000,21,10200,10,3,3,0,117.0
0001-10200-21-128.0,90000,21,10200,10,1,1,128,117.0
0001-10200-45-96.0,80800,45,10200,10,1,1,96,409.0
...,...,...,...,...,...,...,...,...
X290-83150-99-252.0,1846550,99,83150,83,1,1,252,1000.0
X395-83270-265-0.0,1441400,265,83270,83,3,3,0,500.0
X395-83270-265-0.0,1441400,265,83270,83,3,3,0,1500.0


In [15]:
# Séparer les données en variables d'entrée (X) et variable cible (y)
X = data_2021_c.drop("Valeur fonciere", axis=1)
y = data_2021_c["Valeur fonciere"]

In [16]:
#data = data_2021_c.groupby(['ID_Location', 'Date mutation'])

In [17]:
#print(data.head(5))

In [18]:
# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
# Définir une liste d'hyperparamètres à essayer
hyperparameters = [
    {'n_estimators': 3000, 'learning_rate': 0.05, 'max_depth': 7, 'random_state': 50},
    {'n_estimators': 3500, 'learning_rate': 0.05, 'max_depth': 7, 'random_state': 50},
    {'n_estimators': 4000, 'learning_rate': 0.05, 'max_depth': 7, 'random_state': 50},
    {'n_estimators': 4500, 'learning_rate': 0.05, 'max_depth': 7, 'random_state': 50},
    {'n_estimators': 5000, 'learning_rate': 0.05, 'max_depth': 7, 'random_state': 50},
    {'n_estimators': 5500, 'learning_rate': 0.05, 'max_depth': 7, 'random_state': 50},
    {'n_estimators': 6000, 'learning_rate': 0.05, 'max_depth': 7, 'random_state': 50}
]

best_rmse = float('inf')
best_model = None

In [20]:
# Boucle sur les hyperparamètres
for params in hyperparameters:
    # Créer et entraîner le modèle GBM avec les hyperparamètres actuels
    model = GradientBoostingRegressor(**params)
    model.fit(X_train, y_train)

    # Faire des prédictions sur l'ensemble de test
    y_pred = model.predict(X_test)

    # Calculer la racine carrée de l'erreur quadratique moyenne (RMSE)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    # Vérifier si le modèle actuel est le meilleur jusqu'à présent
    if rmse < best_rmse:
        best_rmse = rmse
        best_mae = mae
        best_r2 = r2
        best_model = model

    # Afficher les résultats de l'itération actuelle
    print("Hyperparameters:", params)
    print("RMSE:", rmse)
    # Calcul de la MAE
    print("Mean Absolute Error (MAE):", mae)
    # Calcul du R²
    print("R-squared (R²):", r2)
    print()

Hyperparameters: {'n_estimators': 3000, 'learning_rate': 0.05, 'max_depth': 7, 'random_state': 50}
RMSE: 739015.2118104671
Mean Absolute Error (MAE): 160006.00118486962
R-squared (R²): 0.8807290437483305

Hyperparameters: {'n_estimators': 3500, 'learning_rate': 0.05, 'max_depth': 7, 'random_state': 50}
RMSE: 736125.7124088801
Mean Absolute Error (MAE): 157362.49635340963
R-squared (R²): 0.8816599030992152

Hyperparameters: {'n_estimators': 4000, 'learning_rate': 0.05, 'max_depth': 7, 'random_state': 50}
RMSE: 735043.9685412537
Mean Absolute Error (MAE): 155527.11570875757
R-squared (R²): 0.8820074513869414

Hyperparameters: {'n_estimators': 4500, 'learning_rate': 0.05, 'max_depth': 7, 'random_state': 50}
RMSE: 733861.2466470142
Mean Absolute Error (MAE): 153887.0008917322
R-squared (R²): 0.8823868575265899

Hyperparameters: {'n_estimators': 5000, 'learning_rate': 0.05, 'max_depth': 7, 'random_state': 50}
RMSE: 733290.5384981077
Mean Absolute Error (MAE): 152190.60087994012
R-squared (R

In [21]:
# Afficher les meilleurs résultats
print("Meilleur modèle:")
print("Hyperparameters:", best_model.get_params())
print("RMSE:", best_rmse)
print("Mean Absolute Error (MAE):", best_mae)
print("R-squared (R²):", best_r2)

Meilleur modèle:
Hyperparameters: {'alpha': 0.9, 'ccp_alpha': 0.0, 'criterion': 'friedman_mse', 'init': None, 'learning_rate': 0.05, 'loss': 'squared_error', 'max_depth': 7, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 6000, 'n_iter_no_change': None, 'random_state': 50, 'subsample': 1.0, 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': 0, 'warm_start': False}
RMSE: 731464.5805809569
Mean Absolute Error (MAE): 149783.15434509737
R-squared (R²): 0.8831538121617626
